#  Diagonalize 3d-block of Hamiltonian $H_0$
Find basis (eigenvectors) which diagonalizes the 3d-block of the single-particle Hamiltonian $H_0$.

#### Recommended usage: Copy notebook to (or close to) simulation folder and tailor it to the system of study. 

Choose simulation folder to study. 
Read `Local hamiltonian` from RSPt `out` file.
Diagonalize Hamiltonian and use eigenvectors to transform from spherical harmonics basis to basis where Hamiltonian is diagonal. 
Save transformation vectors in RSPt-friendly format to disk.

In [ ]:
import numpy as np

from py4rspt import rsptt
reload(rsptt)

# Simulation folder to read information from. 
# Update path to desired directory 
foldername = ('path/to/simulations/folder/relative/'
              '2/notebook/position/')
# Number of correlated orbitals
norb = 5  # 5 for d-systems, 7 for f-systems
# If calculations are spin-polarized
spinpol = False
# Label for specifying user defined transformation matrix.
# E.g. name of irreducible representation. 
irrlabel = '1'
# Limit print output to three digits
%precision 3

hs, labels = rsptt.parse_matrices(foldername+'out')
# Loop over all the local Hamiltonians
for i,(h,label) in enumerate(zip(hs,labels)):
    print '------ Cluster label:',label,'------'
    # name of the file to write into
    fwrite = (foldername + '/proj-' 
              + label + '-Irr' + irrlabel + '.inp')
    if spinpol:
        hd = np.copy(h)
    else:
        hd = h[0:norb,0:norb]  # dn block
    print 'Hamiltonian:'
    print 'Real part:'
    print rsptt.print_matrix(hd.real)
    print 'Imag part:'
    print rsptt.print_matrix(hd.imag)
    print
    # Eigenvalues and eigenvectors (column vectors)
    e,v = np.linalg.eigh(hd)    
    # Reorder eigenvectors
    if spinpol:
        vs = np.zeros_like(v)
        es = np.zeros_like(e)
        a,b = 0,norb
        for i in range(2*norb):
            if np.sum(np.abs(v[:norb,i])**2) > 0.5:
                vs[:,a] = v[:,i]
                es[a] = e[i]
                a += 1
            else:
                vs[:,b] = v[:,i]
                es[b] = e[i]
                b += 1
    else:
        vs = np.copy(v)
        es = np.copy(e)
    print "Eigenvalues:"
    print es
    if spinpol:
        print 'Eigenvalues: e(dn)-e(up):'
        print es[:5]-es[5:]
    print "eigenvectors:"
    print 'Real part:'
    print rsptt.print_matrix(vs.real)
    print 'Imag part:'
    print rsptt.print_matrix(vs.imag)  
    print 'Diagonalized Hamiltonian:'
    hdiag = np.dot(np.transpose(np.conj(vs)),np.dot(hd,vs))
    print 'Real part:'
    print rsptt.print_matrix(hdiag.real)
    print 'Imag part:'
    print rsptt.print_matrix(hdiag.imag)
    print
    # Save rotation matrices to file in a RSPt adapted format
    rsptt.write_proj_file(vs, spinpol, fwrite)